# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [2]:
keywords = ['python', 'парсинг', 'unity', 'информационная безопасность'] # Расширел список, так как не в любой момент есть новости с тегами 'python', 'парсинг'

list_data, list_header, list_url = [], [], []
res_dict = {'data': list_data,
         'header': list_header,
         'url': list_url}

url = 'https://habr.com/ru/all/'
web_page = requests.get(url)

if web_page.status_code != 200:
    print(f'Страница {url} недоступна.')
    
else:
    soup = BeautifulSoup(web_page.text, 'html.parser')
    
    news_block = soup.find_all('article', class_='post post_preview') # Блок новости
       
    news_tegs_block = soup.find_all('ul', class_='post__hubs inline-list') # Блок с тегами
    news_tegs = list(map(lambda x: x.find_all('a', class_='inline-list__item-link hub-link'), news_tegs_block)) # Собрали все теги в список (если тегов несколько, то в элементе списка вложенный список)
    
    for i, ls1 in enumerate(news_tegs):
        for ls2 in ls1:
            
            for kw in keywords:
                
                if kw in ls2.get_text().lower():  # Ищем новость по ключевым словам
                
                    news_date = list(map(lambda x: x.find('span', class_='post__time'), news_block))
                    news_header = list(map(lambda x: x.find('a', class_='post__title_link'), news_block))
                    news_url = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), news_block))

                    list_data.append(news_date[i].get_text())
                    list_header.append(news_header[i].get_text())
                    list_url.append(news_url[i])
                
    res_dict['data'] = list_data
    res_dict['header'] = list_header
    res_dict['url'] = list_url
    
res_df = pd.DataFrame(res_dict)
res_df

,data,header,url
0,сегодня в 18:22,Сказ о разработке амбициозного проекта 16-ти л...,https://habr.com/ru/post/513502/
1,сегодня в 16:16,Локальное хранилище или куки? Безопасное хране...,https://habr.com/ru/company/ruvds/blog/512866/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [3]:
import json
import datetime as dt

emails = ['ivan@mail.ru', 'jhon@google.com'] # На какие-то email выдается левая информация, поэтому ниже ее удаляем (пример ivan@mail.ru)

def checking_email (email):
    url = 'https://digibody.avast.com/v1/web/leaks'
    data = {'email': email}
    res_page = requests.post(url, data=json.dumps(data))
    return res_page.json()


list_email, list_date, list_source, list_description = [], [], [], []
res_check_dict = {'email': list_email,
                  'date': list_date,
                  'source': list_source,
                  'description': list_description}

for email in emails:
    result = checking_email(email)
    
    for element in result['value']:
        if email == element['username']: # Чистим
            list_email.append(element['username'])
            list_date.append(dt.datetime.fromtimestamp(element['leak_info']['date']/1000))
            list_source.append(element['leak_info']['title'])
            list_description.append(element['leak_info']['description'])

res_check_dict['email'] = list_email
res_check_dict['date'] = list_date
res_check_dict['source'] = list_source
res_check_dict['description'] = list_description
    
res_check_df = pd.DataFrame(res_check_dict)
res_check_df

,email,date,source,description
0,ivan@mail.ru,2019-02-06 03:00:00,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
1,ivan@mail.ru,2020-06-25 03:00:00,"""Cash Cloud"" USA Combo List","At an unconfirmed date, this ""Cash Cloud"" Comb..."
2,ivan@mail.ru,2020-06-25 03:00:00,Food Services Combo List,"At an unconfirmed date, this Food Services Com..."
3,ivan@mail.ru,2019-02-06 03:00:00,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
4,ivan@mail.ru,2019-02-06 03:00:00,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
5,ivan@mail.ru,2019-02-06 03:00:00,Collection #3 Combo List,"On January 7, 2019, an online user named Sanix..."
6,ivan@mail.ru,2020-01-30 03:00:00,Lekool Games,"At an unconfirmed date, the online gaming site..."
7,ivan@mail.ru,2019-04-04 03:00:00,Game Salad,"In February 2019, Gamesalad's database was all..."
8,ivan@mail.ru,2020-04-23 03:00:00,Aptoide,"In April 2020, the software marketplace app Ap..."
9,ivan@mail.ru,2020-04-09 03:00:00,VK Mix,"At an unconfirmed date, the Russian social med..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
